In [3]:
import numpy as np

a = np.array([1,2,3,4])
a = list(a)
a

[1, 2, 3, 4]

In [3]:
from os.path import join
import numpy as np
import pandas as pd

from feature.dataset import make_dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from category_encoders import *

# train_path = join('jeju_data', 'train_new.parquet')
# test_path = join('jeju_data', 'test_new.parquet')
# holiday_path = join('jeju_data', '국가공휴일.csv')

# sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')

# x_train, y_train, test = make_dataset(train_path, test_path, holiday_path)


In [4]:
train = pd.read_parquet('./jeju_data/train_48.parquet')
test = pd.read_parquet('./jeju_data/test_48.parquet')

In [9]:
# X = train.drop(['target'], axis=1)
# y = train['target']

In [5]:
X = train.drop(['target'], axis=1)
y = train['target']

In [5]:
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

params = {'n_estimators': 4693, 'max_depth': 15, 'min_child_weight': 4, 'gamma': 1, 'learning_rate': 0.018,
          'colsample_bytree': 0.9015018647603987, 'lambda': 1.386941960727803, 'alpha': 0.10534800837686535, 'subsample': 1.0}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1103)

folds = []

for train_idx, val_idx in skf.split(X, y):
    folds.append((train_idx, val_idx))

XGB_model = {}

for f in range(5):
    print(f'===================================={f+1}============================================')
    train_idx, val_idx = folds[f]

    x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[val_idx], y.iloc[train_idx], y.iloc[val_idx]

    XGB = XGBRegressor(**params, tree_method='gpu_hist', gpu_id=0, random_state=1103)
    XGB.fit(x_train, y_train)

    y_pred = XGB.predict(x_val)
    mae = mean_absolute_error(y_val, y_pred)
    print(f"{f + 1} Fold MAE = {mae}")
    XGB_model[f] = XGB
    print(f'================================================================================\n\n')


sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')

for fold in range(5):
    sample_submission['target'] += XGB_model[fold].predict(test)/5
sample_submission.to_csv("./submit_xgb_fold50.csv", index=False)


sample_submission['target'] = round(sample_submission['target'])
sample_submission.to_csv("./submit_xgb_round50.csv", index=False)


c:\mb_job\project\Competition\DACON\jeju_traffic_prediction\jeju_project\lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


====================================1============================================
1 Fold MAE = 2.9561007081009096


====================================2============================================
2 Fold MAE = 2.9444884204119917


====================================3============================================
3 Fold MAE = 2.9419080130559663


====================================4============================================
4 Fold MAE = 2.9549098427006513


====================================5============================================
5 Fold MAE = 2.9451300570806844




In [7]:
df_imp = pd.DataFrame({'imp':XGB.feature_importances_}, index = XGB.feature_names_in_)
df_imp = df_imp[df_imp.imp > 0].sort_values('imp').copy()
df_imp

,imp
day_of_week,0.000580
post_holiday,0.000602
over_max_speed,0.000767
sin_time,0.000780
rest,0.000821
group_time,0.000826
holiday,0.000839
pre_holiday,0.000860
cos_time,0.000886
base_hour,0.000893


--- 49 ---

In [11]:
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

params = {'n_estimators': 4693, 'max_depth': 15, 'min_child_weight': 4, 'gamma': 1, 'learning_rate': 0.018,
          'colsample_bytree': 0.9015018647603987, 'lambda': 1.386941960727803, 'alpha': 0.10534800837686535, 'subsample': 1.0}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1103)

folds = []

for train_idx, val_idx in skf.split(X, y):
    folds.append((train_idx, val_idx))

XGB_model = {}

for f in range(5):
    print(f'===================================={f+1}============================================')
    train_idx, val_idx = folds[f]

    x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[val_idx], y.iloc[train_idx], y.iloc[val_idx]

    XGB = XGBRegressor(**params, tree_method='gpu_hist', gpu_id=0, random_state=1103)
    XGB.fit(x_train, y_train)

    y_pred = XGB.predict(x_val)
    mae = mean_absolute_error(y_val, y_pred)
    print(f"{f + 1} Fold MAE = {mae}")
    XGB_model[f] = XGB
    print(f'================================================================================\n\n')


sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')

for fold in range(5):
    sample_submission['target'] += XGB_model[fold].predict(test)/5
sample_submission.to_csv("./submit_xgb_fold49.csv", index=False)


sample_submission['target'] = round(sample_submission['target'])
sample_submission.to_csv("./submit_xgb_round49.csv", index=False)


c:\mb_job\project\Competition\DACON\jeju_traffic_prediction\jeju_project\lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


====================================1============================================
1 Fold MAE = 2.9538723716533553


====================================2============================================
2 Fold MAE = 2.9428968331481644


====================================3============================================
3 Fold MAE = 2.938217725510741


====================================4============================================
4 Fold MAE = 2.9506707350560415


====================================5============================================
5 Fold MAE = 2.94683139568248




KeyError: 5

--- 48 ---

In [15]:
sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')

for fold in range(5):
    sample_submission['target'] += XGB_model[fold].predict(test)/5
sample_submission.to_csv("./submit_xgb_fold49.csv", index=False)


sample_submission['target'] = round(sample_submission['target'])
sample_submission.to_csv("./submit_xgb_round49.csv", index=False)

randomstate = 42 fold=10

In [13]:
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

params = {'n_estimators': 4693, 'max_depth': 15, 'min_child_weight': 4, 'gamma': 1, 'learning_rate': 0.018,
          'colsample_bytree': 0.9015018647603987, 'lambda': 1.386941960727803, 'alpha': 0.10534800837686535, 'subsample': 1.0}

skf = StratifiedKFold(n_splits=9, shuffle=True, random_state=404)

folds = []

for train_idx, val_idx in skf.split(X, y):
    folds.append((train_idx, val_idx))

XGB_model = {}

for f in range(9):
    print(f'===================================={f+1}============================================')
    train_idx, val_idx = folds[f]

    x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[val_idx], y.iloc[train_idx], y.iloc[val_idx]

    XGB = XGBRegressor(**params, tree_method='gpu_hist', gpu_id=0, random_state=404)
    XGB.fit(x_train, y_train)

    y_pred = XGB.predict(x_val)
    mae = mean_absolute_error(y_val, y_pred)
    print(f"{f + 1} Fold MAE = {mae}")
    XGB_model[f] = XGB
    print(f'================================================================================\n\n')


sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')

for fold in range(9):
    sample_submission['target'] += XGB_model[fold].predict(test)/9
sample_submission.to_csv("./submit_xgb_9fold.csv", index=False)


c:\mb_job\project\Competition\DACON\jeju_traffic_prediction\jeju_project\lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=9.
  warnings.warn(


====================================1============================================
1 Fold MAE = 2.944034164576878


====================================2============================================


KeyboardInterrupt: 

In [12]:
sample_submission['target'] = round(sample_submission['target'])
sample_submission.to_csv("./submit_xgb_round.csv", index=False)

In [6]:
import pandas as pd
sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')

params = {'learning_rate': 0.037114901553206285, 'bagging_temperature': 2.7339987006273825, 'n_estimators': 3064, 'max_depth': 16, 'random_strength': 7, 'colsample_bylevel': 0.40583883059586223, 'l2_leaf_reg': 2.594944843998521e-05, 'min_child_samples': 67, 'max_bin': 262, 'od_type': 'Iter'}

from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

folds = []

for train_idx, val_idx in skf.split(X, y):
    folds.append((train_idx, val_idx))

CAT_model= {}

for f in range(5):
      print(f'===================================={f+1}============================================')
      train_idx, val_idx = folds[f]
      
      x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[val_idx], y.iloc[train_idx], y.iloc[val_idx]
      
      CAT = CatBoostRegressor(**params)
      CAT.fit(x_train, y_train)
      
      y_pred = CAT.predict(x_val)
      mae = mean_absolute_error(y_val, y_pred)
      print(f"{f + 1} Fold MAE = {mae}")
      CAT_model[f] = CAT
      print(f'================================================================================\n\n')
              

for fold in range(5):
    sample_submission['target'] += CAT_model[fold].predict(test)/5    
sample_submission.to_csv("./catcat.csv", index=False)

c:\mb_job\project\Competition\DACON\jeju_traffic_prediction\jeju_project\lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


====================================1============================================
0:	learn: 15.4650871	total: 1.62s	remaining: 1h 22m 47s
1:	learn: 14.9943884	total: 3.02s	remaining: 1h 17m 6s
2:	learn: 14.5424890	total: 4.67s	remaining: 1h 19m 24s
3:	learn: 14.1180205	total: 6.21s	remaining: 1h 19m 11s
4:	learn: 13.7101559	total: 7.7s	remaining: 1h 18m 31s
5:	learn: 13.3079164	total: 9.23s	remaining: 1h 18m 27s
6:	learn: 12.9262085	total: 10.8s	remaining: 1h 18m 52s
7:	learn: 12.5648705	total: 12.3s	remaining: 1h 18m 32s
8:	learn: 12.2180570	total: 13.8s	remaining: 1h 18m 14s
9:	learn: 11.8850777	total: 15.5s	remaining: 1h 18m 59s
10:	learn: 11.5668311	total: 16.9s	remaining: 1h 18m 10s
11:	learn: 11.2644153	total: 18.2s	remaining: 1h 17m 13s
12:	learn: 10.9708016	total: 19.8s	remaining: 1h 17m 37s
13:	learn: 10.6948339	total: 21.6s	remaining: 1h 18m 35s
14:	learn: 10.4287436	total: 23s	remaining: 1h 18m
15:	learn: 10.1748748	total: 24.6s	remaining: 1h 18m
16:	learn: 9.9289235	total: 

In [8]:
sample_submission['target'] = round(sample_submission['target'])
sample_submission.to_csv("./submit_cat_round.csv", index=False)